In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.3.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,455 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,784 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
sql= """
SELECT
YEAR(date) AS YEAR,
ROUND(AVG(price), 2) AS AVG_Price
FROM home_sales
WHERE bedrooms=4
GROUP BY YEAR
"""
spark.sql(sql).show()

+----+---------+
|YEAR|AVG_Price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
sql= """
SELECT
YEAR(date_built) AS YEAR,
ROUND(AVG(price), 2) AS AVG_Price
FROM home_sales
WHERE bedrooms=3 and bathrooms=3
GROUP BY YEAR (date_built)
"""
spark.sql(sql).show()



+----+---------+
|YEAR|AVG_Price|
+----+---------+
|2015| 288770.3|
|2013|295962.27|
|2014|290852.27|
|2012|293683.19|
|2016|290555.07|
|2010|292859.62|
|2011|291117.47|
|2017|292676.79|
+----+---------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
sql="""
SELECT
YEAR(date_built) AS YEAR_BUILT,
ROUND(AVG(price), 2) AS AVG_Price
FROM home_sales
WHERE bedrooms=3
and bathrooms=3
and sqft_living>=2000
and floors=2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
"""
spark.sql(sql).show()



+----------+---------+
|YEAR_BUILT|AVG_Price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

sql="""
SELECT
view,
ROUND(AVG(price), 2) AS AVG_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 35000
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| AVG_Price|
+----+----------+
|   7| 288929.09|
|  51| 788128.21|
|  15| 284907.04|
|  54| 798684.82|
|  11| 280356.07|
|  29| 283881.72|
|  69| 750537.94|
|  42| 289225.45|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3| 284314.53|
|  30| 281085.62|
|  34| 286124.07|
|  59|  791453.0|
|   8| 279099.78|
|  22| 284908.42|
|  28| 285474.25|
|  85|1056336.74|
|  16| 291990.83|
+----+----------+
only showing top 20 rows

--- 1.5974009037017822 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
# Next, we are use SparkSQL to cache our table
# Note: when we use SparkSQL to cache a table, the table is immediately cached (no lazy evaluation).
# When using PySpark it will not be cached until an action is run.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

sql="""
SELECT
view,
ROUND(AVG(price), 2) AS AVG_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 35000
"""
spark.sql(sql).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| AVG_Price|
+----+----------+
|   7| 288929.09|
|  51| 788128.21|
|  15| 284907.04|
|  54| 798684.82|
|  11| 280356.07|
|  29| 283881.72|
|  69| 750537.94|
|  42| 289225.45|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3| 284314.53|
|  30| 281085.62|
|  34| 286124.07|
|  59|  791453.0|
|   8| 279099.78|
|  22| 284908.42|
|  28| 285474.25|
|  85|1056336.74|
|  16| 291990.83|
+----+----------+
only showing top 20 rows

--- 1.0060505867004395 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("p_home_sales")

In [15]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('p_home_sales')

In [16]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView("p_home_sales")

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

sql="""
SELECT
view,
ROUND(AVG(price), 2) AS AVG_Price
FROM p_home_sales
GROUP BY view
HAVING AVG(price) >= 35000
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| AVG_Price|
+----+----------+
|   7| 288929.09|
|  51| 788128.21|
|  15| 284907.04|
|  54| 798684.82|
|  11| 280356.07|
|  29| 283881.72|
|  69| 750537.94|
|  42| 289225.45|
|  73| 752861.18|
|  87| 1072285.2|
|  64| 767036.67|
|   3| 284314.53|
|  30| 281085.62|
|  34| 286124.07|
|  59|  791453.0|
|   8| 279099.78|
|  22| 284908.42|
|  28| 285474.25|
|  85|1056336.74|
|  35| 281767.41|
+----+----------+
only showing top 20 rows

--- 1.2960546016693115 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False